### Part 1. Algorithmic Implementation - BFGS Quasi-Newton Algorithm.

In [1]:
# Install libraries
%matplotlib inline
import numpy as np
from sympy import *
from scipy import stats
from scipy.optimize import line_search
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import time

### 1) func = x1^2 + x2^2 + x3^2      and     x0 = 1,1,1

In [2]:
################################################################################################
#                                        Trivariate Calculation                                #
################################################################################################

# Start at Example:
#x = np.matrix('1;1;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2, x3 = symbols('x1:4')
fn = x1**2 + x2**2 + x3**2

# Lambdify objective function
f = lambdify('x1, x2, x3', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2) 
fnDiffx3 = fn.diff(x3) 
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2], [fnDiffx3]])

# Lambdify
fprime = lambdify('x1, x2, x3', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2) ; fnHessx13 = diff(diff(fn, x1),x3)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) ; fnHessx23 = diff(diff(fn, x2),x3)
fnHessx31 = diff(diff(fn, x3),x1) ; fnHessx32 = diff(diff(fn, x3),x2) ; fnHessx33 = diff(diff(fn, x3),x3) 
fnHess = Matrix([[fnHessx11, fnHessx12, fnHessx13], [fnHessx21, fnHessx22, fnHessx23], [fnHessx31, fnHessx32, fnHessx33]])

# Lambdify
fHess = lambdify('x1, x2, x3', fnHess)

# Calculate Example
#f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))

In [3]:
# Start at
x = np.matrix('1;1;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)) , x.item((2,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0)) , x.item((2,0))))
beta = np.identity(3)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0)) , xold.item((2,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0)) , xnew.item((2,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[1]
 [1]
 [1]]


Iteration:  1
Direction:  [[-2.]
 [-2.]
 [-2.]]
Step Length:  0.5
New Value:  [[ 0.]
 [ 0.]
 [ 0.]]




Local minimum occurs at: [[ 0.]
 [ 0.]
 [ 0.]]
Number of steps: 1
runtime:  0.030027151107788086


### 2) func = x1^2 + 2*x2^2 - 2*x1*x2 - 2*x2  and x0 = 0,0

In [4]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('0;0')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = x1**2 + 2*x2**2 - 2*x1*x2 - 2*x2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Calculate Example
#x = np.matrix('0;0')
#f(x.item((0,0)) , x.item((1,0)))
#np.asmatrix(fprime(x.item((0,0)) , x.item((1,0))))
#np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))

In [5]:
# Start at
x = np.matrix('0;0')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
Q = np.asmatrix(fHess(x.item((0,0)) , x.item((1,0))))
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    step = (-1 * np.transpose(direction) * fprimeVal)/ (np.transpose(direction) * Q * direction)
    
    # Calculate new x
    xnew = xold + step.item((0,0)) * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step.item((0,0)))  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[0]
 [0]]


Iteration:  1
Direction:  [[ 0.]
 [ 2.]]
Step Length:  0.25
New Value:  [[ 0. ]
 [ 0.5]]




Iteration:  2
Direction:  [[ 1. ]
 [ 0.5]]
Step Length:  1.0
New Value:  [[ 1.]
 [ 1.]]




Local minimum occurs at: [[ 1.]
 [ 1.]]
Number of steps: 2
runtime:  0.02001333236694336


### 3) func = 100*(x2 - x1**2)^2 + (1-x1)^2 and x0 = -1.2,1
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [6]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('-1.2;1')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = 100*(x2 - x1**2)**2 + (1-x1)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Define function to execute optimizes step size using line search
# Hardcoded due to shortage of time. Not good!!!!
def opt_func(x):
    return (-x[0] + 1)**2 + 100*(-x[0]**2 + x[1])**2

def grad(x):
    return [-400*x[0]*(-x[0]**2 + x[1]) + 2*x[0] - 2, -200*x[0]**2 + 200*x[1]]

In [7]:
# Start at
x = np.matrix('-1.2;1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy line search function to find ideal step size.
    step = line_search(opt_func, grad, np.squeeze(np.asarray(xold)), np.squeeze(np.asarray(direction)))[0]
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[-1.2]
 [ 1. ]]


Iteration:  1
Direction:  [[ 215.6]
 [  88. ]]
Step Length:  0.000938310275875
New Value:  [[-0.9977003]
 [ 1.0825713]]




Iteration:  2
Direction:  [[ 1.51379751]
 [-3.61038338]]
Step Length:  0.134381034419
New Value:  [[-0.79427463]
 [ 0.59740425]]




Iteration:  3
Direction:  [[ 0.00906143]
 [-0.00221391]]
Step Length:  1.0
New Value:  [[-0.7852132 ]
 [ 0.59519034]]




Iteration:  4
Direction:  [[ 0.25249415]
 [-0.37647776]]
Step Length:  1.0
New Value:  [[-0.53271905]
 [ 0.21871258]]




Iteration:  5
Direction:  [[-0.02545506]
 [ 0.09082232]]
Step Length:  1.0
New Value:  [[-0.5581741]
 [ 0.3095349]]




Iteration:  6
Direction:  [[ 0.12733915]
 [-0.13923766]]
Step Length:  1.0
New Value:  [[-0.43083495]
 [ 0.17029724]]




Iteration:  7
Direction:  [[ 0.40258939]
 [-0.39575624]]
Step Length:  0.27231662474
New Value:  [[-0.32120317]
 [ 0.06252623]]




Iteration:  8
Direction:  [[ 0.05605904]
 [-0.02341039]]
Step Length:  1.0
New 

### 4) func = (x1+x2)^4 + x2^2 and x0 = 2,-2
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [8]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')
fn = (x1+x2)**4 + x2**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Define function to execute optimizes step size using line search
# Hardcoded due to shortage of time. Not good!!!!
def opt_func(x):
    return x[1]**2 + (x[0] + x[1])**4

def grad(x):
    return [4*(x[0] + x[1])**3, 2*x[1] + 4*(x[0] + x[1])**3]

In [9]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('2;-2')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy line search function to find ideal step size.
    step = line_search(opt_func, grad, np.squeeze(np.asarray(xold)), np.squeeze(np.asarray(direction)))[0]
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 2]
 [-2]]


Iteration:  1
Direction:  [[ 0.]
 [ 4.]]
Step Length:  0.25
New Value:  [[ 2.]
 [-1.]]




Iteration:  2
Direction:  [[-2.66666667]
 [ 1.44444444]]
Step Length:  1.0
New Value:  [[-0.66666667]
 [ 0.44444444]]




Iteration:  3
Direction:  [[ 0.45379657]
 [-0.43073776]]
Step Length:  1.0
New Value:  [[-0.21287009]
 [ 0.01370669]]




Iteration:  4
Direction:  [[ 0.02435805]
 [-0.01441054]]
Step Length:  1.0
New Value:  [[-0.18851204]
 [-0.00070385]]




Iteration:  5
Direction:  [[ 0.02262781]
 [-0.00606077]]
Step Length:  1.0
New Value:  [[-0.16588423]
 [-0.00676462]]




Iteration:  6
Direction:  [[ 0.0380656 ]
 [-0.00198341]]
Step Length:  1.0
New Value:  [[-0.12781863]
 [-0.00874803]]




Iteration:  7
Direction:  [[ 0.02924645]
 [ 0.00448451]]
Step Length:  1.0
New Value:  [[-0.09857217]
 [-0.00426352]]




Iteration:  8
Direction:  [[ 0.02158077]
 [ 0.00398319]]
Step Length:  1.0
New Value:  [[-0.0769914 ]
 [-0.00028033]]




Iteration:  9

### 5.1) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 1
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [10]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=1
fn = (x1-1)**2 + (x2-1)**2 + 1*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Define function to execute optimizes step size using line search
# Hardcoded due to shortage of time. Not good!!!!
def opt_func(x):
    return (x[0] - 1)**2 + (x[1] - 1)**2 + (x[0]**2 + x[1]**2 - 0.25)**2

def grad(x):
    return [4*x[0]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[0] - 2, 4*x[1]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[1] - 2]

In [11]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy line search function to find ideal step size.
    step = line_search(opt_func, grad, np.squeeze(np.asarray(xold)), np.squeeze(np.asarray(direction)))[0]
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]


Iteration:  1
Direction:  [[ -7.]
 [ 11.]]
Step Length:  0.159187734115
New Value:  [[-0.11431414]
 [ 0.75106508]]




Iteration:  2
Direction:  [[ 1.33354567]
 [ 0.78422954]]
Step Length:  0.225736194404
New Value:  [[ 0.18671539]
 [ 0.92809407]]




Iteration:  3
Direction:  [[ 0.33963704]
 [-0.2678212 ]]
Step Length:  1.0
New Value:  [[ 0.52635243]
 [ 0.66027286]]




Iteration:  4
Direction:  [[ 0.0076482 ]
 [-0.07432081]]
Step Length:  1.0
New Value:  [[ 0.53400063]
 [ 0.58595205]]




Iteration:  5
Direction:  [[ 0.03855413]
 [-0.02303126]]
Step Length:  1.0
New Value:  [[ 0.57255476]
 [ 0.56292079]]




Iteration:  6
Direction:  [[-0.00865956]
 [ 0.00137346]]
Step Length:  1.0
New Value:  [[ 0.5638952 ]
 [ 0.56429425]]




Iteration:  7
Direction:  [[ 0.00018783]
 [-0.00019757]]
Step Length:  1.0
New Value:  [[ 0.56408302]
 [ 0.56409669]]




Iteration:  8
Direction:  [[  4.00017301e-06]
 [ -9.66944918e-06]]
Step Length:  1.0
New Value: 

### 5.2) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 10
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [12]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=10
fn = (x1-1)**2 + (x2-1)**2 + 10*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Define function to execute optimizes step size using line search
# Hardcoded due to shortage of time. Not good!!!!
def opt_func(x):
    return (x[0] - 1)**2 + (x[1] - 1)**2 + (x[0]**2 + x[1]**2 - 0.25)**2

def grad(x):
    return [4*x[0]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[0] - 2, 4*x[1]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[1] - 2]

In [13]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy line search function to find ideal step size.
    step = line_search(opt_func, grad, np.squeeze(np.asarray(xold)), np.squeeze(np.asarray(direction)))[0]
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]


Iteration:  1
Direction:  [[-70.]
 [ 74.]]
Step Length:  0.0187169808839
New Value:  [[-0.31018866]
 [ 0.38505659]]




Iteration:  2
Direction:  [[ 1.94731642]
 [ 1.92280155]]
Step Length:  0.262807222639
New Value:  [[ 0.20158016]
 [ 0.89038272]]




Iteration:  3
Direction:  [[-0.38718028]
 [-0.44747975]]
Step Length:  0.176264583848
New Value:  [[ 0.13333399]
 [ 0.81150789]]




Iteration:  4
Direction:  [[-0.0120808 ]
 [-0.15042183]]
Step Length:  1.0
New Value:  [[ 0.12125319]
 [ 0.66108606]]




Iteration:  5
Direction:  [[ 0.03301813]
 [-0.08343608]]
Step Length:  1.0
New Value:  [[ 0.15427132]
 [ 0.57764998]]




Iteration:  6
Direction:  [[ 0.05713202]
 [-0.06126857]]
Step Length:  1.0
New Value:  [[ 0.21140333]
 [ 0.51638142]]




Iteration:  7
Direction:  [[ 0.09291494]
 [-0.06098325]]
Step Length:  1.0
New Value:  [[ 0.30431827]
 [ 0.45539816]]




Iteration:  8
Direction:  [[ 0.11708506]
 [-0.06181889]]
Step Length:  1.0
New Value

C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

### 5.3) func = (x1-1)^2 + (x2-1)^2 + c*(x1^2 + x2^2 - 0.25)^2 and x0 = 1,-1 at c = 100
#### Notice this is a polynomial problem. Finding the Step size will be challenging as we cannot use Q anymore

In [14]:
################################################################################################
#                                        Bivariate Calculation                                 #
################################################################################################

# Start at Example:
#x = np.matrix('2,-2')

########################################## Objective ###########################################
# Define and symbolize objective function 
x1, x2= symbols('x1:3')

# at C=100
fn = (x1-1)**2 + (x2-1)**2 + 100*(x1**2 + x2**2 - 0.25)**2

# Lambdify objective function
f = lambdify('x1, x2', fn)

########################################## Derivative ###########################################

# Derivative
fnDiffx1 = fn.diff(x1)
fnDiffx2 = fn.diff(x2)  
fnDeriv = Matrix([[fnDiffx1], [fnDiffx2]])

# Lambdify
fprime = lambdify('x1, x2', fnDeriv)

########################################## Hessian    ###########################################

# Hessian
fnHessx11 = diff(diff(fn, x1),x1) ; fnHessx12 = diff(diff(fn, x1),x2)
fnHessx21 = diff(diff(fn, x2),x1) ; fnHessx22 = diff(diff(fn, x2),x2) 
fnHess = Matrix([[fnHessx11, fnHessx12], [fnHessx21, fnHessx22]])

# Lambdify
fHess = lambdify('x1, x2', fnHess)

# Define function to execute optimizes step size using line search
# Hardcoded due to shortage of time. Not good!!!!
def opt_func(x):
    return (x[0] - 1)**2 + (x[1] - 1)**2 + (x[0]**2 + x[1]**2 - 0.25)**2

def grad(x):
    return [4*x[0]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[0] - 2, 4*x[1]*(x[0]**2 + x[1]**2 - 0.25) + 2*x[1] - 2]

In [15]:
# Here step size will be calculated in a different way.
# Start at
x = np.matrix('1;-1')
print ("Initial Starting Point at:", x)

# Set convergence Precision and iteration limit
e = pow(10,-8)
itMax = 1000

# Assigning initial values
xold = np.matrix('0;0')
xnew = x
xlist, ylist = [xnew], [f(x.item((0,0)) , x.item((1,0)))]
beta = np.identity(2)

# Loop Condition for next iteration
fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

# Calculate runtime
starttime = time.time()

# Formula for BFGS
while fprimeNorm/(1 + f_abs) > e:
    xold = xnew
    fprimeVal = np.asmatrix(fprime(xold.item((0,0)) , xold.item((1,0))))
        
    # Halting at itMax iterations to avoid convergence failure and computational overhead.    
    if (len(xlist) == itMax) :
        break;
    
    # Calculate direction
    direction = -1 * np.linalg.inv(beta) * fprimeVal
    
    # Calculate step size
    # Use scipy line search function to find ideal step size.
    step = line_search(opt_func, grad, np.squeeze(np.asarray(xold)), np.squeeze(np.asarray(direction)))[0]
    
    # Calculate new x
    xnew = xold + step * direction
    
    # Update Secant Estimates for next iteration
    s = xnew - xold
    
    fprimeValNew = np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0))))
    y = fprimeValNew - fprimeVal
    
    beta_del = ((-1*(beta*s)*np.transpose(beta*s) / (np.transpose(s)*beta*s))) + ((y*np.transpose(y))/(np.transpose(y)*s))
    beta = beta + beta_del
    
    # Print search Dir, step length, new value: For first 10 and last 5 iterations
    if (len(xlist) <= 10):
        print("\n")
        print("Iteration: ",len(xlist))    
        print("Direction: ",direction)    
        print("Step Length: ",step)  
        print("New Value: ",xnew)
        print("\n")
        
    if (len(xlist) > 10): 
        print(".", end = "")    
    
    xlist.append(xnew)
    ylist.append(f(xnew.item((0,0)) , xnew.item((1,0))))
    
    # Loop Condition for next iteration
    fprimeNorm = np.linalg.norm(np.asmatrix(fprime(xnew.item((0,0)) , xnew.item((1,0)))))
    f_abs = abs(f(xnew.item((0,0)) , xnew.item((1,0))))

runtime = time.time() - starttime 
print("\n")

if (len(xlist) == itMax):
    print ("No Solution has been found after %s iterations" %itMax)
    
else:
    print ("Local minimum occurs at:", xnew)
    print ("Number of steps:", len(xlist)-1)

print ("runtime: ", runtime)

Initial Starting Point at: [[ 1]
 [-1]]


Iteration:  1
Direction:  [[-700.]
 [ 704.]]
Step Length:  0.0018559013606
New Value:  [[-0.29913095]
 [ 0.30655456]]




Iteration:  2
Direction:  [[ 2.07814497]
 [ 2.10668785]]
Step Length:  0.266726385923
New Value:  [[ 0.25516515]
 [ 0.86846379]]




Iteration:  3
Direction:  [[-0.56405462]
 [-0.54794564]]
Step Length:  0.124995391278
New Value:  [[ 0.18466092]
 [ 0.79997311]]




Iteration:  4
Direction:  [[-0.07672339]
 [-0.14936038]]
Step Length:  0.191437620249
New Value:  [[ 0.16997318]
 [ 0.77137992]]




Iteration:  5
Direction:  [[-0.05802437]
 [-0.16148874]]
Step Length:  0.14203483232
New Value:  [[ 0.16173169]
 [ 0.74844289]]




Iteration:  6
Direction:  [[-0.04360804]
 [-0.15632767]]
Step Length:  0.0762166426106
New Value:  [[ 0.15840803]
 [ 0.73652812]]




Iteration:  7
Direction:  [[-0.03597905]
 [-0.1552752 ]]
Step Length:  0.054322894442
New Value:  [[ 0.15645355]
 [ 0.72809312]]




Iteration:  8
Direction:  [[-0.0307314

C:\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:285: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'